<a href="https://colab.research.google.com/github/archiebenn/BIOLM0050_kaggle/blob/master/protein_embedding_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch transformers sentencepiece h5py

In [2]:
import h5py
import numpy as np
import pandas as pd
import os
import torch

from transformers import T5EncoderModel, T5Tokenizer

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"


## workflow aim:
protein sequence → truncate to shorten but keep N and C terminals for localisation info → ProtT5 embedding → single vector → classifier (other script)


## Load ProtT5

In [3]:
# set tokeniser
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# using half prott5
tokeniser = T5Tokenizer.from_pretrained(
    "Rostlab/prot_t5_xl_half_uniref50-enc",
    do_lower_case=False
)

model = T5EncoderModel.from_pretrained(
    "Rostlab/prot_t5_xl_half_uniref50-enc"
).to(device)

model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/196 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


T5EncoderModel(
  (shared): Embedding(128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=4096, bias=False)
              (k): Linear(in_features=1024, out_features=4096, bias=False)
              (v): Linear(in_features=1024, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=16384, bias=False)
              (wo): Linear(in_features=16384, out_features=1024, bias=False)
              (dropout): Dropo

In [4]:
# import training csv
from google.colab import files
uploaded = files.upload()


Saving train.csv to train (1).csv


In [5]:
# data setup
df_train = pd.read_csv("train.csv")

print(f'Train DF length: {len(df_train)}')
df_train.head(10)


Train DF length: 16077


,Id,acc,partition,cytoplasm,nucleus,extracellular,cell_surface,mitochondrion,endom,sequence,...,aa_frac_M,aa_frac_N,aa_frac_P,aa_frac_Q,aa_frac_R,aa_frac_S,aa_frac_T,aa_frac_V,aa_frac_W,aa_frac_Y
0,0,P61966,0,0,0,0,0,0,1,MMRFMLLFSRQGKLRLQKWYLATSDKERKKMVRELMQVVLARKPKM...,...,0.051,0.013,0.013,0.044,0.063,0.057,0.019,0.063,0.013,0.044
1,1,Q9VTK2,0,0,0,0,0,0,1,MSATYTNTITQRRKTAKVRQQQQHQWTGSDLSGESNERLHFRSRST...,...,0.028,0.032,0.044,0.043,0.068,0.080,0.063,0.059,0.025,0.038
2,2,O95858,3,0,0,0,1,0,1,MPRGDSEQVRYCARFSYLWLKFSLIIYSTVFWLIGALVLSVGIYAE...,...,0.034,0.041,0.031,0.027,0.044,0.044,0.051,0.078,0.014,0.058
3,3,Q9WUX5,0,1,0,0,0,0,1,MGRSLTCPFGISPACGAQASWSIFGVGTAEVPGTHSHSNQAAAMPH...,...,0.023,0.036,0.089,0.051,0.050,0.117,0.044,0.058,0.008,0.011
4,4,Q9NQC3-3,1,0,0,0,0,0,1,MDGQKKNWKDKVVDLLYWRDIKKTGVVFGASLFLLLSLTVFSIVSV...,...,0.015,0.030,0.015,0.035,0.035,0.070,0.035,0.101,0.015,0.040
5,6,O00204,3,1,1,0,0,0,1,MDGPAEPQIPGLWDTYEDDISEISQKLPGEYFRYKGVPFPVGLYSL...,...,0.025,0.022,0.104,0.038,0.055,0.093,0.044,0.041,0.025,0.033
6,7,O35095,1,1,0,0,0,0,1,MSCCDLAAAGQLGKAGIMASDCEPALNQAESRNPTLERYLGALREA...,...,0.016,0.018,0.062,0.049,0.053,0.062,0.051,0.058,0.015,0.016
7,8,Q7KRW1,1,0,0,0,0,0,1,MSVRTKVLGLVDVMMRVPPVMVIDEILKMDMGMQSWLYPDKDKASG...,...,0.049,0.026,0.043,0.038,0.047,0.099,0.063,0.077,0.017,0.030
8,10,O45405,1,1,0,0,0,0,1,MPTFSLWLPAGSNNVHPCGDPYAHHLFMRCLYHAKHDPTMKFDVKT...,...,0.025,0.034,0.046,0.021,0.059,0.059,0.076,0.059,0.008,0.029
9,11,Q9H4L5,0,1,1,0,1,0,1,MMSDEKNLGVSQKLVSPSRSTSSCSSKQGSRQDSWEVVEGLRGEMN...,...,0.019,0.046,0.042,0.041,0.061,0.127,0.033,0.053,0.019,0.027


## subsetting df and checking for repeats

In [40]:
# keep just the Id and sequence in the data frame
df_seq_id = df_train[["Id", "sequence"]]
df_seq_id.head()

# Get the counts of each ID
id_counts = df_seq_id["Id"].value_counts()

# checking for any duplicate ids:
df_seq_id["Id"].value_counts().head(50)

# How many IDs are duplicated (appear more than once)
num_duplicated_ids = (id_counts > 1).sum()
print("Number of duplicated IDs:", num_duplicated_ids)

# group by Id and count unique sequences per ID
seq_per_id = df_seq_id.groupby("Id")["sequence"].nunique()

# how many Ids have >1 unique sequence?
num_ids_with_multiple_sequences = (seq_per_id > 1).sum()
print("IDs with multiple sequences:", num_ids_with_multiple_sequences)



Number of duplicated IDs: 2425
IDs with multiple sequences: 0


### So lots of duplicated rows with same sequences, therefore can drop any after first occurance of these:

In [41]:
# remove any occurance of same Id after first
df_seq_id = df_seq_id.drop_duplicates(subset="Id", keep="first")

# sanity check for any duplicate ids:
df_seq_id["Id"].value_counts().head(10)

,count
Id,
16740,1
0,1
1,1
2,1
3,1
4,1
16721,1
16720,1
16719,1


In [29]:
# testing for longest sequence in train.csv:
# Compute string lengths
df_train["sequence_lengths"] = df_train["sequence"].astype(str).str.len()

# sort in new df by seq length
df_sorted = df_train.sort_values("sequence_lengths", ascending=False)

df_sorted["sequence_lengths"].head(25)

,sequence_lengths
5598,34350
11943,18562
2346,18141
10752,13100
8148,8886
4725,8545
7632,8081
9759,7570
15527,7570
193,7388


## embedding protein function

In [19]:
def embed_protein(seq):

  # reformat sequences to be space separated AAs:
  seq = ' '.join(list(seq))

  inputs = tokeniser(seq, return_tensors="pt")
  inputs = {k: v.to(device) for k, v in inputs.items()}

  with torch.no_grad():
    outputs = model(**inputs)

  emb = outputs.last_hidden_state.squeeze(0).cpu().numpy()

  return emb.mean(axis=0)


### Chunking and pooling sequences
As some of the sequences are fa too large (up to 34k AAs) for ProtT5, will attempt to chunk and pool the sequences into ProtT5 instead:

## embedding chunked protein function


In [13]:
from IPython.terminal.embed import embed
def embed_chunky_protein(seq, chunk_size=1024, overlap=50):
    """
    embed a protein sequence in chunks and pool into a single fixed-length vector.
    Returns tensor of shape [hidden_dim].
    """
    seq_str = str(seq)

    # Skip empty sequences
    if len(seq_str) == 0:
        return None

    # Chunk sequence
    chunks = []
    start = 0
    while start < len(seq_str):
        end = min(start + chunk_size, len(seq_str))
        chunks.append(seq_str[start:end])
        start += chunk_size - overlap

    # embed each chunk and mean pool
    chunk_vector = []
    for chunk in chunks:
      try:
            emb = embed_protein(chunk)
            if emb is not None:

                # convert emb to a tensor
                chunk_vector.append(torch.tensor(emb))

      except Exception as e:
            print(f"Skipping a chunk due to error: {e}")
      finally:
            # free GPU memory if needed
            if 'emb' in locals():
                del emb
            torch.cuda.empty_cache()

    if len(chunk_vector) == 0:
        return None

    # mean pool over chunks to get one vector per protein:
    protein_vector = torch.mean(torch.stack(chunk_vector), dim=0)
    return protein_vector.cpu().numpy()

## Truncating sequences
Chunking and pooling taking too long and could contain unhelpful information contained within the middle of the sequences. Instead will try to keep the 1024AAs at start and end (longest seq = 2048AAs) as this will likely contain the most useful 'postcode' information.

In [22]:
# truncate function to 128aa either end
def truncate_prot(seq, chunk_len = 2048):
  seq = str(seq)
  if len(seq) <= chunk_len*2:

    # short seq, return whole
    return seq

  else:
    # take first 1024 and last 1024 amino acids of sequence
    return seq[:chunk_len] + seq[-chunk_len:]


In [37]:
# just adding to shorten dataset for testing
#df_seq_id = df_seq_id.head(100)

df_seq_id["sequence_lengths"] = df_seq_id["sequence"].astype(str).str.len()

df_seq_id_sorted = df_seq_id.sort_values("sequence_lengths", ascending=False)

df_seq_id_sorted["sequence_lengths"].head(25)

,sequence_lengths
5598,34350
11943,18562
2346,18141
10752,13100
8148,8886
4725,8545
7632,8081
9759,7570
193,7388
5742,7158


In [24]:
# write out the .h5 file:
# this is using the half protT5 model and truncated to a maximum length og 4096aa (first and last 2048aa based off testing longer and shorter lengths with the given GPU memory)

with h5py.File("protT5_half_embeddings_2048aa.h5", "w") as f:
  for _, row in df_seq_id.iterrows():

    # 1. set prot_id as a string for h5py to work (for h5 file column)
    prot_id = str(row["Id"])

    # check if dataset already exists to handle duplicate IDs in input data
    if prot_id in f:
        print(f"Skipping {prot_id}, dataset already exists in the file (duplicate ID in input data).")
        continue

    # 2. run embed protein function on each sequence:
    seq = row["sequence"]

    # truncate protein to max. 256AAs
    seq = truncate_prot(seq)

    try:
            emb = embed_protein(seq)

            if emb is None:
                print(f"Skipping {prot_id}, empty sequence")
                continue

            # convert to fixed numeric type for HDF5
            emb = np.array(emb, dtype=np.float32)

            # write to HDF5
            f.create_dataset(prot_id, data=emb)

    except RuntimeError as e:
            print(f"Skipping {prot_id} due to runtime error: {e}")

    finally:
            # free GPU memory
            if 'emb' in locals():
                del emb
            torch.cuda.empty_cache()